In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import math 

#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [2]:
if not os.path.exists("images"):
    os.mkdir("images")

day = "31"
month = "07"
dataset = "task"
date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day

folder_path = 'data_fabrica/'
#folder_path = 'data_lab/'
def process_file(date):
    file_path = os.path.join(folder_path,f'{dataset}_{date}.csv')
    if os.path.exists(file_path):

        df = pd.read_csv(file_path, parse_dates=['ts'])

        # Remove first empty column if it exists
        if df.shape[1] == 5:
            df = df.drop(df.columns[0], axis=1)
        # Remove entries with 'sec' equal to 0
        df= df[df['sec'] != 0]

        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)
        # Reset the index of the DataFrame
        df = df.reset_index(drop = True)

        # Create a new column 'task_label' based on the mapping
        task_label_mapping = {0: 'Break', 1: 'TEKOX Red Black Cables', 2: 'TEKOX Blue Brown Cables', 3: 'MFALG Assembly',
                            4: 'Screws in Boards', 5: 'Extra activity 1', 6: 'Extra Activity 2', 7: 'Unkown Activity'}
        df['task_name'] = df['task'].map(task_label_mapping)


        emp_label_mapping = {0: 'Fernando', 1: 'Antonio', 2: 'Daniela', 3: 'Diogo', 4: 'Elisabete',5: 'Isabel', 6: 'Joao', 
                            7:'Joaquim',8:'Lidia',9:'Maria',10:'Mario',11:'Patricia',12:'Raquel',13:'Ana',14:'Paulo',15:'Luis'}
        df['emp_name'] = df['empid'].map(emp_label_mapping)
        #display(df)
    else:
        return pd.DataFrame
    return df


df = process_file(date)
#display(df)

In [3]:
def remove_outliers(df,alpha):
    ref_av_time = np.array([-1,12.0, 12, 144, 40, 60, -1, -1])

    ref_av_time[0] = df[df['task'] == 0]['sec'].mean()
    #ref_av_time[5] = df[df['task'] == 5]['sec'].mean()
    ref_av_time[6] = df[df['task'] == 6]['sec'].mean()
    ref_av_time[7] = df[df['task'] == 7]['sec'].mean()

    my_med_time = np.zeros(8)
    my_av_time = np.zeros(8)
    for task_id in range(1,8):
        my_med_time[task_id] = df[df['task'] == task_id]['sec'].median()
        my_av_time[task_id] = df[df['task'] == task_id]['sec'].mean()
    print(my_med_time)
    
    print("Reference average")
    print(ref_av_time)
    outliers = 0
    use_ref = 0
    print("Outliers:")
    for index, row in df.iterrows():
        for task_id in range(1,8):
                print()
                if ((row['task'] == task_id) and ((row['sec'] < (1 - alpha) * my_med_time[task_id]) or (row['sec'] > (2) * my_av_time[task_id]))):
                    outliers += 1
                    #print(row['task_name'],row['sec'])
                    df = df.drop(index=index)
    
    mean_and_variance(df)
    print()
    print(f"Outliers removed: {outliers}")
    return df

def mean_and_variance(df):

    av_time = np.zeros(8)
    var_time = np.zeros(8)
    med_time = np.zeros(8)
    count_task = np.zeros(8)
    for task_id in range(0,8):
        count_task[task_id] = df.loc[df['task'] == task_id].shape[0]
        task_data = df[df['task'] == task_id]['sec']
        sum_task = task_data.sum()
        av_time[task_id] = task_data.mean()
        med_time[task_id] = task_data.median()
        var_time[task_id] = task_data.var()
        
        if not math.isnan(av_time[task_id]):
            print(f"Task: {task_id}")
            print(f"Average:{round(av_time[task_id],2)}")
            print(f"Median:{round(med_time[task_id],2)}")
        #if not math.isnan(var_time[task_id]):
            print(f"Variance:{round(var_time[task_id],2)}")        
            std_dev = np.sqrt(var_time[task_id])
            print(f"Standard deviation: {round(std_dev,2)}")
            print(f"Count: {count_task[task_id]}")
            print(f"total secs: {sum_task}")
            count_per_hour = 3600*count_task[task_id]/sum_task
            print(f"Count per hour: {round(count_per_hour)}")
            print()
    return 


df1 = remove_outliers(df, alpha = 0.5)
#display(df1)

[ 0. nan nan nan 26. nan nan nan]
Reference average
[580.57142857  12.          12.         144.          40.
  60.                  nan          nan]
Outliers:









































































































































































































































































































































































































































































































































































































































































































































































































































































In [4]:
include_break_pie = 0

def plot_task(date_year,df):

    color_discrete_task = { 'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue', 'MFALG Assembly': 'gold',
                      'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange', 'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}
    
    color_discrete_emp = {'Fernando': 'cornflowerblue', 'Luis': 'yellowgreen'}

    column_order = ['ts', 'empid','emp_name','task','task_name','sec']
    df = df[column_order]
    # Exclude task '0' from the DataFrame
    if not include_break_pie:
        df1 = df[df['task'] != 0]
    else:
        df1 = df
    #display(df1)
    #TASK
    # Sum the 'sec' values based on the 'task' column
    sum_by_task = df1.groupby('task_name')['sec'].sum().reset_index()

    for index,row in sum_by_task.iterrows():
        print(row['task_name'],row['sec'])
        sum_task_dt =  pd.Timedelta(seconds = row['sec'])
        print(sum_task_dt)

    # Create a pie chart
    fig_task = px.pie(sum_by_task, names='task_name', 
                      values='sec',
                      color = 'task_name',
                      color_discrete_map = color_discrete_task, 
                      width = 600)
                      #title='Time Spent in Each Task')
    fig_task.show()
    fig_task.write_image('images/taskpie_' + date + '.svg')



    #EMPLOYEE // do not work in break
    df2 = df1[df1['task'] != 0]
    # Sum the 'sec' values based on the 'emp_name' column
    sum_by_emp = df2.groupby('emp_name')['sec'].sum().reset_index()
    #print(sum_by_emp['sec'])
    #print(sum_by_emp)
    for index,row in sum_by_emp.iterrows():
        print(row['emp_name'],row['sec'])
        sum_emp_dt =  pd.Timedelta(seconds = row['sec'])
        print(sum_emp_dt)

    # Create a pie chart
    fig_emp = px.pie(sum_by_emp, names='emp_name', 
                     values='sec', 
                     color = 'emp_name',
                      color_discrete_map = color_discrete_emp,
                      width = 550)
                      #title='Time Worked by Each Employee')
    fig_emp.show()

    fig_emp.write_image('images/emppie_' + date + '.svg')
plot_task(date_year,df1)

#for i in unique_values:
#    total_time[i] = 
    

Screws in Boards 12448
0 days 03:27:28


OSError: [Errno 22] Invalid argument: 'images\\taskpie_31_07.svg'

In [ ]:
include_break_bar = 0
def plot_tasks_in_time(date_year,df):
    #display(px.data.medals_wide())
    # Create a dictionary to map models to colors
    #task_label_mapping = {0: 'Break', 1: 'TEKOX Red Black Cables', 2: 'TEKOX Blue Brown Cables', 3: 'MFALG Assembly',
    #                  4: 'Screws in Boards', 5: 'Extra activity 1', 6: 'Extra Activity 2', 7: 'Unkown Activity'}
    if not include_break_bar:
        df = df[df['task'] != 0]
    else:
        df = df
    #df = df[df['task_name'] == 'Screws in Boards']
    #display(df)

    color_discrete = { 'Break': 'navy', 'TEKOX Red Black Cables': 'tomato', 'TEKOX Blue Brown Cables': 'cornflowerblue', 'MFALG Assembly': 'gold',
                      'Screws in Boards': 'darkgray', 'Extra activity 1': 'darkorange', 'Extra Activity 2': 'violet', 'Unkown Activity': 'limegreen'}

    # Create bars for the first subplot
    fig = px.bar(df, x="ts", y="sec",
                color="task_name",
                labels={"ts": "Timestamp", "sec": "Time (seconds)",'task_name': 'Task names'},
                color_discrete_map = color_discrete)
                #itle='Tasks Performed Over Time')


    # Get the minimum and maximum values of the y-axis
    y_max = df['sec'].max()

    # Set the y-axis range from the minimum to the maximum + 1
    fig.update_layout(yaxis_range=[0, y_max + 1], xaxis_range = [date_year + ' 09:00:00', date_year + ' 18:00:00'])#y_max + 1])
    fig.update_layout(bargap=0) # gap between bars of adjacent location coordinates.
    fig.update_traces(width=25000)

    #bargroupgap=0.1) # gap between bars of the same location coordinate.)
    #display(df)
    
    fig.show()



plot_tasks_in_time(date_year,df1)

c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



This section will now gather a range of days and present the corresponding graphs


In [ ]:
start_date = '31/07/2023'
end_date = '27/09/2023'

#interesting break screws mfalg
#start_date = '01/08/2023'
#end_date = '02/08/2023'

#not very int -> extra activity 1 Fernando
#start_date = '06/09/2023'
#end_date = '07/09/2023'


start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
end_date = pd.to_datetime(end_date, format='%d/%m/%Y')

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

dfs = []
df_all = pd.DataFrame()

def all_plots(date_year,df):
            
    plot_task(date_year,df)
    plot_tasks_in_time(date_year,df)


    return

def plots_all_dates(date_range):
    
    print_v = 0
    for date in date_range:

        day = str(date.day).zfill(2)
        month = str(date.month).zfill(2)
        date_ = day + "_" + month
        print(date_)
        date_year = "2023-" + month + "-" + day

        df = process_file(date_)
        if df.empty:
            print("No data in that date")
            continue            

        df1 = remove_outliers(df, alpha = 0.5)
        
        all_plots(date_year,df1)
        dfs.append(df1)

        #df_for_date = get_data_for_date(date)

    return dfs

dfs = plots_all_dates(date_range)
df_all = pd.concat(dfs, ignore_index=True)
print("1-2")
all_plots(date_year,df_all)




31_07
Reference average
[580.57142857  12.          12.         144.          40.
  60.                  nan          nan]
Outliers:
Screws in Boards 2
Screws in Boards 1
Screws in Boards 1
Screws in Boards 2
Screws in Boards 1
Screws in Boards 396
Screws in Boards 76
Screws in Boards 585
Screws in Boards 78
Screws in Boards 644
Screws in Boards 623
Screws in Boards 69
Screws in Boards 416
Screws in Boards 67
Screws in Boards 88
Task: 0
Average:580.57
Median:166.0
Variance:1406300.95
Standard deviation: 1185.88
Count: 7.0
total secs: 4064
Count per hour: 6

Task: 4
Average:27.42
Median:26.0
Variance:37.1
Standard deviation: 6.09
Count: 454.0
total secs: 12448
Count per hour: 131


Outliers removed: 15
Screws in Boards 12448
0 days 03:27:28


Fernando 12448
0 days 03:27:28


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



01_08
Reference average
[467.33333333  12.          12.         144.          40.
  60.                  nan          nan]
Outliers:
Screws in Boards 153
Screws in Boards 59
Task: 0
Average:467.33
Median:292.0
Variance:303956.33
Standard deviation: 551.32
Count: 3.0
total secs: 1402
Count per hour: 8

Task: 3
Average:117.56
Median:112.0
Variance:281.53
Standard deviation: 16.78
Count: 45.0
total secs: 5290
Count per hour: 31

Task: 4
Average:26.58
Median:25.0
Variance:24.83
Standard deviation: 4.98
Count: 90.0
total secs: 2392
Count per hour: 135


Outliers removed: 2
MFALG Assembly 5290
0 days 01:28:10
Screws in Boards 2392
0 days 00:39:52


Fernando 7682
0 days 02:08:02


...

Luis 1478
0 days 00:24:38


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



1-2
Extra activity 1 16047
0 days 04:27:27
MFALG Assembly 18883
0 days 05:14:43
Screws in Boards 15533
0 days 04:18:53
TEKOX Blue Brown Cables 325
0 days 00:05:25
TEKOX Red Black Cables 460
0 days 00:07:40


Fernando 49770
0 days 13:49:30
Luis 1478
0 days 00:24:38


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

